<a href="https://colab.research.google.com/github/YouMinJung/Music_Creator_With_LSTM/blob/main/Music_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Feb  7 22:56:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install music21

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd "midi_songs_dataset"
!unzip -qq "midi_songs_dataset.zip"
%cd ..

/content/midi_songs_dataset
/content


## **music creator code**

In [4]:
import glob, pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream 
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
midi = converter.parse("midi_songs_dataset/moz1.mid")

notes_to_parse = midi.flat.notes
print(np.shape(notes_to_parse))
for e in notes_to_parse[:20]:
  print(e, e.offset)

(11857,)
<music21.chord.Chord G3 B-3> 8.0
<music21.note.Note G> 8.0
<music21.note.Note G> 8.0
<music21.chord.Chord G3 B-3> 8.5
<music21.chord.Chord B-3 G4> 9.0
<music21.chord.Chord B-3 G4> 9.5
<music21.chord.Chord G3 B-3> 10.0
<music21.chord.Chord G3 B-3> 10.5
<music21.note.Note E-> 11.0
<music21.note.Note E-> 11.0
<music21.chord.Chord B-3 G4> 11.0
<music21.note.Note D> 11.5
<music21.note.Note D> 11.5
<music21.chord.Chord B-3 G4> 11.5
<music21.note.Note D> 12.0
<music21.note.Note D> 12.0
<music21.chord.Chord B-3 G3> 12.0
<music21.note.Note G> 12.0
<music21.note.Note G> 12.0
<music21.chord.Chord B-3 G3> 12.5


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [5]:
notes = []
for i,file in enumerate(glob.glob("midi_songs_dataset/*.mid")):
  midi = converter.parse(file) 
  
  notes_to_parse = None
  try:
    parts = instrument.partitionByInstrument(midi)
  except TypeError:
    print('{} file occur error.'.format(file))

  if parts:
    notes_to_parse = parts.parts[0].recurse() 
  else :
    notes_to_parse = midi.flat.notes

  for e in notes_to_parse:
    if isinstance(e, note.Note):
      notes.append(str(e.pitch))
    elif isinstance(e, chord.Chord):
      notes.append('.'.join(str(n) for n in e.normalOrder))

print('complete to parse files')

complete to parse files


In [6]:
seqence_len = 100

n_vocab = (len(set(notes)))
print('Classes of notes : ', n_vocab, '\n')

print('Notes : ', notes[:10])
print('Length of notes : ', len(notes), '\n') 

pitchnames = sorted(set(item for item in notes))
print('Pitchnames : ', pitchnames[:10])
print('Length of pitchnames : ', len(pitchnames), '\n')

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print('Note To Int : ', note_to_int,  '\n')

net_in = []
net_out = []

for i in range(0, len(notes) - seqence_len):
  seq_in = notes[i:i + seqence_len] 
  seq_out = notes[i + seqence_len]
  
  net_in.append([note_to_int[char] for char in seq_in])
  net_out.append(note_to_int[seq_out])
  
print('shape of network_in : ', np.shape(net_in))
print('shape of network_out : ', np.shape(net_out))

n_patterns = len(net_in)
print('n_patterns : ', n_patterns)

net_in = np.reshape(net_in, (n_patterns, seqence_len, 1))
print('shape of net_in : ', net_in.shape)

net_in = net_in / float(n_vocab)
net_out = np_utils.to_categorical(net_out)
print('shape of net_out : ', net_out.shape)

Classes of notes :  213 

Notes :  ['G5', 'D5', 'E-5', 'F#4', 'G4', 'B-5', 'A5', 'G5', 'F#5', 'G5']
Length of notes :  72984 

Pitchnames :  ['0', '0.2', '0.2.4.7', '0.2.5', '0.2.5.7', '0.2.7', '0.3', '0.3.5', '0.3.6', '0.3.6.9']
Length of pitchnames :  213 

Note To Int :  {'0': 0, '0.2': 1, '0.2.4.7': 2, '0.2.5': 3, '0.2.5.7': 4, '0.2.7': 5, '0.3': 6, '0.3.5': 7, '0.3.6': 8, '0.3.6.9': 9, '0.4': 10, '0.4.7': 11, '0.5': 12, '0.6': 13, '1': 14, '1.2': 15, '1.4': 16, '1.4.7': 17, '1.4.7.9': 18, '1.4.8': 19, '1.6': 20, '1.7': 21, '10': 22, '10.0': 23, '10.0.2': 24, '10.1': 25, '10.1.4': 26, '10.11': 27, '10.2': 28, '10.3': 29, '11': 30, '11.0': 31, '11.0.4': 32, '11.0.4.7': 33, '11.1': 34, '11.1.4': 35, '11.2': 36, '11.2.4': 37, '11.2.4.7': 38, '11.2.5': 39, '11.2.5.7': 40, '11.2.6': 41, '11.3': 42, '11.4': 43, '2': 44, '2.3': 45, '2.4': 46, '2.4.5.9': 47, '2.4.7': 48, '2.4.8': 49, '2.4.9': 50, '2.5': 51, '2.5.7': 52, '2.5.8': 53, '2.5.8.11': 54, '2.5.9': 55, '2.6': 56, '2.6.7': 57, '2.6

In [7]:
model = Sequential(name="Music_Creator_LSTM")
model.add(CuDNNLSTM(512, input_shape=(net_in.shape[1], net_in.shape[2]), return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(CuDNNLSTM(512))
model.add(Dense(256))
model.add(Dropout(rate=0.3))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "Music_Creator_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 100, 512)          1054720   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 100, 512)          2101248   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 512)               2101248   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)        

In [8]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint] 

model.fit(net_in, net_out, epochs=70, batch_size=64, callbacks=callbacks_list)

Epoch 1/70
1139/1139 [==============================] - 98s 79ms/step - loss: 4.1791
Epoch 2/70
1139/1139 [==============================] - 90s 79ms/step - loss: 4.0604
Epoch 3/70
1139/1139 [==============================] - 90s 79ms/step - loss: 4.0063
Epoch 4/70
1139/1139 [==============================] - 90s 79ms/step - loss: 3.9179
Epoch 5/70
1139/1139 [==============================] - 90s 79ms/step - loss: 3.7855
Epoch 6/70
1139/1139 [==============================] - 90s 79ms/step - loss: 3.5896
Epoch 7/70
1139/1139 [==============================] - 90s 79ms/step - loss: 3.3281
Epoch 8/70
1139/1139 [==============================] - 90s 79ms/step - loss: 3.0321
Epoch 9/70
1139/1139 [==============================] - 90s 79ms/step - loss: 2.7218
Epoch 10/70
1139/1139 [==============================] - 91s 80ms/step - loss: 2.4206
Epoch 11/70
1139/1139 [==============================] - 91s 80ms/step - loss: 2.1338
Epoch 12/70
1139/1139 [==============================] - 91s 80

In [9]:
net_in = []
output = []

for i in range(0, len(notes) - seqence_len, 1):
  seq_in = notes[i:i + seqence_len]
  seq_out = notes[i + seqence_len]
  
  net_in.append([note_to_int[char] for char in seq_in])
  output.append(note_to_int[seq_out])
  
n_patterns = len(net_in)

In [10]:
start_seqence = np.random.randint(0, len(net_in)-1)
pattern = net_in[start_seqence]
print('Random Sequence : ', pattern)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print('int_to_note : ', int_to_note)

Random Sequence :  [16, 198, 174, 173, 210, 143, 198, 186, 198, 51, 210, 143, 204, 162, 161, 128, 86, 204, 128, 147, 168, 167, 153, 133, 147, 210, 147, 12, 153, 133, 170, 168, 167, 133, 12, 147, 133, 170, 168, 167, 153, 110, 147, 10, 210, 110, 198, 12, 133, 186, 110, 152, 84, 133, 84, 12, 84, 133, 0, 132, 12, 132, 133, 84, 12, 84, 148, 133, 171, 0, 0, 154, 110, 148, 10, 211, 110, 199, 12, 133, 187, 110, 23, 199, 197, 133, 84, 96, 133, 96, 197, 147, 84, 148, 143, 143, 143, 176, 185, 176]
int_to_note :  {0: '0', 1: '0.2', 2: '0.2.4.7', 3: '0.2.5', 4: '0.2.5.7', 5: '0.2.7', 6: '0.3', 7: '0.3.5', 8: '0.3.6', 9: '0.3.6.9', 10: '0.4', 11: '0.4.7', 12: '0.5', 13: '0.6', 14: '1', 15: '1.2', 16: '1.4', 17: '1.4.7', 18: '1.4.7.9', 19: '1.4.8', 20: '1.6', 21: '1.7', 22: '10', 23: '10.0', 24: '10.0.2', 25: '10.1', 26: '10.1.4', 27: '10.11', 28: '10.2', 29: '10.3', 30: '11', 31: '11.0', 32: '11.0.4', 33: '11.0.4.7', 34: '11.1', 35: '11.1.4', 36: '11.2', 37: '11.2.4', 38: '11.2.4.7', 39: '11.2.5', 4

In [11]:
prediction_output = []

for i in range(0, 250):
  pred_in = np.reshape(pattern, (1, len(pattern), 1))
  pred_in = pred_in / float(n_vocab)

  prediction = model.predict(pred_in, verbose=0)
  index = np.argmax(prediction)
  result = int_to_note[index]
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

  print('\r', 'Predicted Notes ', i, " ",result, end='')

 Predicted Notes  249   7.11

In [12]:
offset = 0 
output_notes = []

for pattern in prediction_output:
    
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        
        notes = [] 
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) 
            new_note.storedInstrument = instrument.Piano() 
            notes.append(new_note) 

        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)

    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output/Creator_Output.mid')

'output/Creator_Output.mid'

In [13]:
model.save('Music_Creator_LSTM_lastver.h5')
del model

## **Using Pre-Trained Model**

In [18]:
from keras.models import load_model

model = load_model('weights-improvement-34-0.3004-bigger.hdf5')
#model = load_model('Music_Creator_LSTM_lastver.h5')

In [19]:
start_seqence = np.random.randint(0, len(net_in)-1)
pattern = net_in[start_seqence]
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

prediction_output = []

for i in range(0, 500):
  pred_in = np.reshape(pattern, (1, len(pattern), 1))
  pred_in = pred_in / float(n_vocab)

  prediction = model.predict(pred_in, verbose=0)
  index = np.argmax(prediction)
  result = int_to_note[index]
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

offset = 0 
output_notes = []

for pattern in prediction_output:
    
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        
        notes = [] 
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) 
            new_note.storedInstrument = instrument.Piano() 
            notes.append(new_note) 

        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)

    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output/Creator_Output.mid')

'output/Creator_Output.mid'